# Uso de filtros espaciales y morfológicos

In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'cv2'

In [ ]:
image_unir = cv.imread('im/unir-1.jpg')

In [ ]:
def mask(n):
    '''Define a nxn mask.
    Useful for a quick filter
    '''
    mask = np.ones((n,n), np.uint8)
    return mask

kernel = mask(5)

im = image_unir

erosion_5 = cv.erode(im, kernel, iterations=1)
dilation_5 = cv.dilate(im, kernel, iterations=1)
cv.imwrite('out/erosion_5.jpg', erosion_5)
cv.imwrite('out/dilation_5.jpg', dilation_5)

In [ ]:
im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
erosion_5 = cv.cvtColor(erosion_5, cv.COLOR_BGR2RGB)
dilation_5 = cv.cvtColor(dilation_5, cv.COLOR_BGR2RGB)
plt.subplot(131)
plt.imshow(im)
plt.title('Imagen Original'), plt.xticks([]), plt.yticks([])
plt.subplot(132)
plt.imshow(erosion_5)
plt.title('Erosión 5x5'), plt.xticks([]), plt.yticks([])
plt.subplot(133)
plt.imshow(dilation_5)
plt.title('Dilatación 5x5'), plt.xticks([]), plt.yticks([])
plt.show()